In [1]:
import os
import itertools
import PyPDF2  #library used for converting pdf to text
import docx2txt #library used for translating docx to text

import joblib
from joblib import Parallel, delayed #to make it easy to write readable parallel code and debug it quickly

import re #to check if a string contains the specified search pattern

import logging #tracking events that happen when some software runs

import nltk #it provides text processing libraries


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

import spacy #for information extraction and natural language understanding
english_nlp = spacy.load('en_core_web_sm')
import en_core_web_sm #en (english), core (vocabulary, syntax, entities), web (written text), sm (size-12MB)
from spacy.matcher import Matcher #The Matcher lets you find words and phrases using rules describing their token attributes
from spacy.matcher import PhraseMatcher
nlp = en_core_web_sm.load()
designitionmatcher = PhraseMatcher(nlp.vocab)
from nltk.corpus import stopwords #To remove stopwords as part of text preprocessing

STOPWORDS = set(stopwords.words('english'))
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 
            'ME', 'M.E', 'M.E.', 'M.B.A', 'MBA', 'MS', 'M.S', 
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH', 
            'SSLC', 'SSC' 'HSC', 'CBSE', 'ICSE', 'X', 'XII' , 
            'Pre-University' , 'Engineering' , 'Central Board of Secondary Education'
        ]

class resumeparsing():

    def sub_control(self,text):
        print("Email-->", self.find_email(text))
        print("Skills-->", self.find_skills(text))
        print("Phone number-->", self.find_phone(text))
        print("First name-->", self.find_firstname(text))
        print("Full name-->", self.find_fullname(text))
        print("Experience -->", self.find_experience(text))
        print("Designation-->", self.find_designition(text))
        print("Education-->", self.find_education(text))

    def iteratingfile(self):
        directory = 'C:\\Users\\admin\\Desktop\\Ceipal US resume files'
        for filename in os.listdir(directory):  
            f = os.path.join(directory, filename)
            if os.path.isfile(f):
                if f.endswith(".pdf"):
                    text = self.pdf_to_text(f)
                    self.sub_control(text)
                if f.endswith(".docx"):
                    text =  self.doc_to_text(f)
                    self.sub_control(text)

    def pdf_to_text (self,filepath):
        try:
            pdfFileObj = open(filepath, 'rb') 
            pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
            x = pdfReader.numPages
            text = ''
            for i in range(x):
                pages = pdfReader.getPage(i)
                text += pages.extractText()                    
            return text
        except:
            pass
    
    def doc_to_text (self,filepath):
        try:
            text = ''
            text += docx2txt.process(filepath)
            return text
        except:
            pass

    def find_email(self,text):
        try:
            r = re.compile(r'[\w\.-]+@[\w\.-]+')
            return r.findall(text)
        except:
            pass

    def get_skills(self,text,nlp_from_joblib):
        try:
            doc = nlp_from_joblib(text)
            myset = []
            subset = []
            for ent in doc.ents:
                if ent.label_ == "SKILL":
                    subset.append(ent.text)
            myset.append(subset)
            return subset
        except:
            pass

    def find_skills(self,text):
        try :
            nlp_from_joblib = joblib.load('C:\\Users\\admin\\Desktop\\Skills Extraction\\resumefinal.pkl')
            nlp_from_joblib(text)
            resume_skills = list(set(self.get_skills(text.lower(),nlp_from_joblib)))
            return resume_skills
        except:
            pass

    def find_phone(self,text):
        try:
            return list(iter(phonenumbers.PhoneNumberMatcher(text, None)))[0].raw_string
        except:
            try:
                return re.search(
                    r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})',
                    text).group()
            except:
                return ""

    def find_firstname(self,text):
        try:
            person_names = []
 
            for sent in nltk.sent_tokenize(text):
                for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
                    if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                        person_names.append(
                        ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                        )
 
            return person_names[0]
        except:
            pass

    def find_fullname(self,text):
        try:
            spacy_parser = english_nlp(text)
            return(spacy_parser.ents[0])
        except:
            pass
    
    def find_experience(self,text):
        try:
            def correct_year(self,text):
                if len(text) < 2:
                    if int(text) > int(str(date.today().year)[-2:]):
                        text = str(int(str(date.today().year)[:-2]) - 1) + text
                    else:
                        text = str(date.today().year)[:-2] + text
                return text

  
            experience = 0
            start_month = -1
            start_year = -1
            end_month = -1
            end_year = -1

            not_alpha_numeric = r'[^a-zA-Z\d]'
            number = r'(\d{2})'

            months_num = r'(01)|(02)|(03)|(04)|(05)|(06)|(07)|(08)|(09)|(10)|(11)|(12)'
            months_short = r'(jan)|(feb)|(mar)|(apr)|(may)|(jun)|(jul)|(aug)|(sep)|(oct)|(nov)|(dec)'
            months_long = r'(january)|(february)|(march)|(april)|(may)|(june)|(july)|(august)|(september)|(october)|(november)|(december)'
            month = r'(' + months_num + r'|' + months_short + r'|' + months_long + r')'
            regex_year = r'((20|19)(\d{2})|(\d{2}))'
            year = regex_year
            start_date = month + not_alpha_numeric + r"?" + year
        
            end_date = r'((' + number + r'?' + not_alpha_numeric + r"?" + month + not_alpha_numeric + r"?" + year + r')|(present|current|till date|today))'
            longer_year = r"((20|19)(\d{2}))"
            year_range = longer_year + r"(" + not_alpha_numeric + r"{1,4}|(\s*to\s*))" + r'(' + longer_year + r'|(present|current|till date|today))'
            date_range = r"(" + start_date + r"(" + not_alpha_numeric + r"{1,4}|(\s*to\s*))" + end_date + r")|(" + year_range + r")"

        
            regular_expression = re.compile(date_range, re.IGNORECASE)
        
            regex_result = re.search(regular_expression, text)

            while regex_result:
          
                try:
                    date_range = regex_result.group()
                    try:
              
                        year_range_find = re.compile(year_range, re.IGNORECASE)
                        year_range_find = re.search(year_range_find, date_range)
               
                        replace = re.compile(r"((\s*to\s*)|" + not_alpha_numeric + r"{1,4})", re.IGNORECASE)
                        replace = re.search(replace, year_range_find.group().strip())
         
                        start_year_result, end_year_result = year_range_find.group().strip().split(replace.group())
       
                        start_year_result = int(correct_year(start_year_result))
                        if (end_year_result.lower().find('present') != -1 or 
                            end_year_result.lower().find('current') != -1 or 
                            end_year_result.lower().find('till date') != -1 or 
                            end_year_result.lower().find('today') != -1): 
                            end_month = date.today().month  # current month
                            end_year_result = date.today().year  
                        else:
                            end_year_result = int(correct_year(end_year_result))

                    except Exception as e:
            
                        start_date_find = re.compile(start_date, re.IGNORECASE)
                        start_date_find = re.search(start_date_find, date_range)

                        non_alpha = re.compile(not_alpha_numeric, re.IGNORECASE)
                        non_alpha_find = re.search(non_alpha, start_date_find.group().strip())

                        replace = re.compile(start_date + r"(" + not_alpha_numeric + r"{1,4}|(\s*to\s*))", re.IGNORECASE)
                        replace = re.search(replace, date_range)
                        date_range = date_range[replace.end():]
        
                        start_year_result = start_date_find.group().strip().split(non_alpha_find.group())[-1]

           
                        start_year_result = int(correct_year(start_year_result))

                        if date_range.lower().find('present') != -1 or date_range.lower().find('current') != -1:
                            end_month = date.today().month 
                            end_year_result = date.today().year 
                        else:
                            end_date_find = re.compile(end_date, re.IGNORECASE)
                            end_date_find = re.search(end_date_find, date_range)

                            end_year_result = end_date_find.group().strip().split(non_alpha_find.group())[-1]

                            try:
                                end_year_result = int(correct_year(end_year_result))
                            except Exception as e:
                                logging.error(str(e))
                                end_year_result = int(re.search("\d+",correct_year(end_year_result)).group())

                    if (start_year == -1) or (start_year_result <= start_year):
                        start_year = start_year_result
                    if (end_year == -1) or (end_year_result >= end_year):
                        end_year = end_year_result

                    text = text[regex_result.end():].strip()
                    regex_result = re.search(regular_expression, text)
                except Exception as e:
                    logging.error(str(e))
                    text = text[regex_result.end():].strip()
                    regex_result = re.search(regular_expression, text)
            
                return end_year - start_year
        except:
            pass
                    
    def get_experience(self,text):
        try:
            total_exp = 0
            if len(text['work_and_employment'].keys()):
                text = ""
                for key, values in text['work_and_employment'].items():
                    text += " ".join(values) + " "
                    total_exp = resumeparse.calculate_experience(text)
                    return total_exp, text
            else:
                text = ""
                for key in text.keys():
                    if key != 'education_and_training':
                        if key == 'contact_info':
                            text += " ".join(text[key]) + " "
                        else:
                            for key_inner, value in restextme_segments[key].items():
                                text += " ".join(value) + " "
                total_exp = resumeparse.calculate_experience(text)
                return total_exp, text
            return total_exp, " "
        except:
            pass
        
    # def getExperience(self,text,infoDict,debug=False):
    #     experience=[]
    #     try:
    #         for sentence in self.lines:#find the index of the sentence where the degree is find and then analyse that sentence
    #                 sen=" ".join([words[0].lower() for words in sentence]) #string of words in sentence
    #                 if re.search('experience',sen):
    #                     sen_tokenised= nltk.word_tokenize(sen)
    #                     tagged = nltk.pos_tag(sen_tokenised)
    #                     entities = nltk.chunk.ne_chunk(tagged)
    #                     for subtree in entities.subtrees():
    #                         for leaf in subtree.leaves():
    #                             if leaf[1]=='CD':
    #                                 experience=leaf[0]
    #     except Exception as e:
    #         print traceback.format_exc()
    #         print e 
    #     if experience:
    #         infoDict['experience'] = experience
    #     else:
    #         infoDict['experience']=0
    #     if debug:
    #         print "\n", print(infoDict), "\n"
    #         code.interact(local=locals())
    #     return experience


    def find_designition(self,text):
        try:
            job_titles = []
            nlp = en_core_web_sm.load()
        
            __nlp = nlp(text.lower())
        
            matches = designitionmatcher(__nlp)
            for match_id, start, end in matches:
                span = __nlp[start:end]
                job_titles.append(span.text)
            return job_titles
        except:
            pass

    def find_education(self,text):
        try:
            nlp_text = nlp(text)
            nlp_text = [sent.text.strip() for sent in nlp_text.sents]

            edu = {}
            for index, text in enumerate(nlp_text):
                for tex in text.split():
                    tex = re.sub(r'[?|$|.|!|,]', r'', tex)
                    if tex.upper() in EDUCATION and tex not in STOPWORDS:
                        edu[tex] = text + nlp_text[index + 1]
            education = []
            for key in edu.keys():
                year = re.search(re.compile(r'(((20|19)(\d{})))'), edu[key])
                if year:
                    education.append((key, ''.join(year[0])))
                else:
                    education.append(key)
            return education
        except:
            pass

rp1 = resumeparsing()

text = rp1.iteratingfile()
print(text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


Email--> ['Swiftk401@gmail.com']
Skills--> ['windows', 'software', 'email marketing', 'databases', 'component', 'debugging', 'business process', 'database', 'support', 'tracking system', 'framework', 'data structures', 'deployment', 'languages', 'design', 'testing', 'business', 'bash', 'web service', 'data integration', 'command line interface', 'git', 'access control', 'metadata', 'javascript', 'schedule', 'library', 'correctness', 'amp', 'security', 'user interface', 'java', 'visual studio code', 'eclipse', 'operating systems', 'postman', 'workflow', 'marketing']
Phone number--> 737-520-6996
First name--> Karen


ERROR:root:'NoneType' object has no attribute 'group'


Full name--> Karen Swift

Sr.
Experience --> 0
Designation--> []
Education--> ['MS']
Email--> ['harish.oracle@yandex.com']
Skills--> ['queue', 'engineering', 'software', 'oracle', 'databases', 'business process', 'mobile computing', 'component', 'database', 'support', 'mobile', 'accounting', 'monitoring', 'languages', 'design', 'testing', 'business', 'linux', 'functional testing', 'release', 'git', 'cross validation', 'computer science', 'content management', 'library', 'finance', 'compass', 'publishing', 'payments', 'shell', 'github', 'workflow', 'specification']
Phone number--> 732-344-3609
First name--> Career Overview


ERROR:root:correct_year() missing 1 required positional argument: 'text'


...

Full name--> 25 Jan 2022
Experience --> 0
Designation--> []
Education--> []
Email--> ['Sparsh.goel1909@gmail.com']
Skills--> ['apache tomcat', 'junit', 'engineering', 'oracle', 'databases', 'box', 'server', 'documentation', 'database', 'hibernate', 'framework', 'deployment', 'design', 'testing', 'business', 'mysql', 'front end', 'release', 'schedule', 'javascript', 'computer science']
Phone number--> 352-745-5717


ERROR:root:'NoneType' object has no attribute 'group'


First name--> Logical
Full name--> SPARSH GOEL                                            IAM SailPoint
Experience --> 0
Designation--> []
Education--> []
Email--> ['odumedelphine@gmail.com']
Skills--> ['windows', 'tableau', 'release', 'framework', 'finance', 'support', 'software', 'shippable', 'telecommunications', 'computer science', 'server', 'documentation', 'confluence', 'business', 'unity', 'linux', 'ios', 'chrome']
Phone number--> 904-416-8252


ERROR:root:'NoneType' object has no attribute 'end'


First name--> Agile Champion
Full name--> 904
Experience --> 0
Designation--> []
Education--> ['MS']
Email--> ['kottekumar611@gmail.com']
